In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
import numpy as np

# Load the dataset and preprocess
data = pd.read_csv('/content/HCD(Preprocessed) (1).csv')

In [3]:
# Replace current loan status with binary values
data['Current_loan_status'] = data['Current_loan_status'].replace({'NO DEFAULT': 0, 'DEFAULT': 1})

data['historical_default'] = data['historical_default'].replace({'N': 'temp', 'Unknown': 'N'}).replace({'temp': 'Unknown'})

<ipython-input-3-fad2bf2e336d>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Current_loan_status'] = data['Current_loan_status'].replace({'NO DEFAULT': 0, 'DEFAULT': 1})


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28638 entries, 0 to 28637
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   customer_id          28638 non-null  float64
 1   customer_age         28638 non-null  int64  
 2   customer_income      28638 non-null  float64
 3   home_ownership       28638 non-null  object 
 4   employment_duration  28638 non-null  float64
 5   loan_intent          28638 non-null  object 
 6   loan_grade           28638 non-null  object 
 7   loan_amnt            28638 non-null  float64
 8   loan_int_rate        28638 non-null  float64
 9   term_years           28638 non-null  int64  
 10  historical_default   28638 non-null  object 
 11  cred_hist_length     28638 non-null  int64  
 12  Current_loan_status  28638 non-null  int64  
dtypes: float64(5), int64(4), object(4)
memory usage: 2.8+ MB


In [ ]:
data.tail()

,customer_id,customer_age,customer_income,home_ownership,employment_duration,loan_intent,loan_grade,loan_amnt,loan_int_rate,term_years,historical_default,cred_hist_length,Current_loan_status
28633,32577.0,57,53000.0,MORTGAGE,1.0,PERSONAL,C,5800.0,13.16,7,N,30,0
28634,32578.0,54,120000.0,MORTGAGE,4.0,PERSONAL,A,17625.0,7.49,4,N,19,0
28635,32579.0,65,76000.0,RENT,3.0,HOMEIMPROVEMENT,B,35000.0,10.99,5,Unknown,28,1
28636,32580.0,56,150000.0,MORTGAGE,5.0,PERSONAL,B,15000.0,11.48,6,N,26,0
28637,32581.0,99,42000.0,RENT,2.0,MEDICAL,B,6475.0,9.99,6,N,30,0


In [4]:


# Encode categorical features
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    if column != 'Current_loan_status':  # Skip target column
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])
        label_encoders[column] = le

X = data.drop('Current_loan_status', axis=1)
y = data['Current_loan_status']

# Resample the data
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Train the model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)




/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [19:23:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [5]:
from sklearn.metrics import classification_report, accuracy_score

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)

# Generate the classification report
report = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy Score: {accuracy:.4f}')
print('Classification Report:')
print(report)


Accuracy Score: 0.9804
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      6764
           1       0.98      0.98      0.98      6822

    accuracy                           0.98     13586
   macro avg       0.98      0.98      0.98     13586
weighted avg       0.98      0.98      0.98     13586



In [ ]:
y_test.head()

,Current_loan_status
6100,0
12301,0
24588,0
3418,0
29724,1


In [16]:
def calculate_credit_score(user_data):
    """
    Calculates the credit score for a given user's data within a range of 300 to 850
    and assigns a credit score category.

    Parameters:
    user_data (dict): Dictionary containing user details.

    Returns:
    tuple: Integer credit score and category.
    """
    # Initialize the base score
    X = 300  # Starting range of the score

    # Payment History Score based only on historical_default
    payment_history_score = 200 if user_data['historical_default'] == 'N' else 50

    # Credit Utilization Score
    credit_utilization_ratio = min(user_data['loan_amnt'] / user_data['customer_income'], 1)
    credit_utilization_score = 150 * (1 - credit_utilization_ratio)

    # Credit History Length Score
    credit_history_score = min(user_data['cred_hist_length'] * 20, 100)

    # Employment Stability Score
    employment_stability_score = min(int(user_data['employment_duration']) / 12 * 10, 100)

    # Loan Grade Score
    loan_grade_scores = {'A': 100, 'B': 80, 'C': 60, 'D': 40, 'E': 20, 'F': 10}
    loan_grade_score = loan_grade_scores.get(user_data['loan_grade'], 10)

    # Calculate the total score
    raw_credit_score = X + (payment_history_score + credit_utilization_score +
                            credit_history_score + employment_stability_score +
                            loan_grade_score)

    # Rescale the credit score to the 300–850 range
    min_raw_score = 360  # Minimum raw score possible with the original scale
    max_raw_score = 950  # Maximum raw score possible with the original scale
    credit_score = int(((raw_credit_score - min_raw_score) / (max_raw_score - min_raw_score)) * 550 + 300)

    # Determine the credit score category
    if credit_score >= 800:
        category = 'Excellent'
    elif credit_score >= 740:
        category = 'Very Good'
    elif credit_score >= 670:
        category = 'Good'
    elif credit_score >= 580:
        category = 'Fair'
    else:
        category = 'Poor'

    return credit_score, category


In [27]:
def predict_loan_default_and_score(user_data):
    """
    Predicts loan default and calculates credit score.

    Parameters:
    user_data (dict): A dictionary containing user details. Keys should match the feature names.

    Returns:
    tuple: Prediction ('DEFAULT' or 'NO DEFAULT') and credit score.
    """
    # Create a DataFrame from the user data
    input_data = pd.DataFrame([user_data])

    # Encode categorical variables using the saved label encoders
    for column in label_encoders:
        if column in input_data.columns:
            if input_data[column].iloc[0] not in label_encoders[column].classes_:
                input_data[column] = np.nan  # Handle unknown categories
            else:
                input_data[column] = label_encoders[column].transform(input_data[column])
        else:
            input_data[column] = np.nan  # Fill with NaN if the column is missing

    # Ensure all data types are correct (convert to float)
    input_data = input_data.fillna(0)  # Fill NaNs with 0 (or a default value)
    input_data = input_data.astype(float)  # Ensure all input data is float

    # Make prediction
    prediction = xgb_model.predict(input_data)
    prediction_result = 'DEFAULT' if prediction[0] == 1 else 'NO DEFAULT'

    # Calculate credit score
    credit_score, credit_category = calculate_credit_score(user_data)

    return prediction_result, credit_score, credit_category

# Example usage
user_info = {
    'customer_id': 301,
    'customer_age': 50,
    'customer_income': 120000,          # High income
    'home_ownership': 'OWN',
    'employment_duration': '12',        # 12 years of employment
    'loan_intent': 'HOMEIMPROVEMENT',
    'loan_grade': 'A',                  # High loan grade
    'loan_amnt': 10000,                 # Low loan amount relative to income
    'loan_int_rate': 5.0,
    'term_years': 3,
    'historical_default': 'N',          # No history of default
    'cred_hist_length': 10              # Long credit history (10 years)
}

result, credit_score, credit_category = predict_loan_default_and_score(user_info)
print("Prediction:", result)
print("Credit Score:", credit_score)
print("Credit Category:", credit_category)

Prediction: NO DEFAULT
Credit Score: 754
Credit Category: Very Good
